In [2]:
!pip install geopandas

     |████████████████████████████████| 972kB 5.8MB/s 
     |████████████████████████████████| 6.5MB 8.0MB/s 
     |████████████████████████████████| 14.8MB 328kB/s 


In [1]:
#Importing libraries
import pandas as pd
import geopandas as gpd
import numpy as np

In [12]:
#Importing crime data and opening the weather data file:
!wget https://data.buffalony.gov/api/views/d6g9-xbgu/rows.csv?accessType=DOWNLOAD;
#Reading the crime data into a pandas dataframe :
crime_df = pd.read_csv('/content/rows.csv?accessType=DOWNLOAD')
#converting the incident_datetime column into datetime format:
crime_df['incident_datetime']=pd.to_datetime(crime_df['incident_datetime'])
# Sorting the data by time :
crime_df.sort_values('incident_datetime',inplace=True)
# Creating a new neighborhood column with empty values :
crime_df['neighborhood']=np.nan
crime_df['police_dt']=np.nan
crime_df['council_dt']=np.nan

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
#Reading the geojson data provided by https://data.buffalony.gov "https://data.buffalony.gov/Economic-Neighborhood-Development/Neighborhoods/q9bk-zu3p" :
nbh_polygon = gpd.read_file("/content/Neighborhoods.geojson")
pd_polygon = gpd.read_file("/content/BPD Districts.geojson")
cd_polygon = gpd.read_file("/content/Council Districts.geojson")

# Converting the pandas dataframe into a geopandas dataframe(has higher support for geospatial data) :
gdf = gpd.GeoDataFrame( crime_df, geometry=gpd.points_from_xy(crime_df.longitude, crime_df.latitude))

In [14]:
pd_polygon.head()

,bdy_left,name,st_prefix,rhs,st_code,hi_x_name,city_left,fromleft,lo_x_pre,direction,objectid,globalid,lhs,fromright,bdy_right,st_type,hi_x_type,lo_x_name,toright,lo_x_type,city_right,st_suffix,hi_x_suf,street,recnum_r,toleft,shape_stlength,lo_x_suf,shape_starea,recnum_l,hi_x_pre,sbe4_id,geometry
0,None,District B,None,None,None,None,None,0,None,None,3,{F6B399B4-524B-45CF-83A0-06C06BCCA628},None,0,None,None,None,None,0,None,None,None,None,None,0,0,62635.398738778073,None,152849720.51699948,0,None,0,"MULTIPOLYGON (((-78.90197 42.89666, -78.90113 ..."
1,None,District E,None,None,None,None,None,0,None,None,2,{83F2A179-549E-40B5-ACC4-5A0ED9DBF195},None,0,None,None,None,None,0,None,None,None,None,None,0,0,71419.982740536478,None,188176251.96103966,0,None,1,"MULTIPOLYGON (((-78.83168 42.94597, -78.83023 ..."
2,None,District C,None,None,None,None,None,0,None,None,4,{323FC3F5-857C-4E2E-A95D-4A8CC6112099},None,0,None,None,None,None,0,None,None,None,None,None,0,0,58574.689856890174,None,178912266.02432826,0,None,0,"MULTIPOLYGON (((-78.83762 42.87883, -78.83794 ..."
3,None,District A,None,None,None,None,None,0,None,None,1,{FF525ECB-B916-4644-9F18-A71150F86DAD},None,0,None,None,None,None,0,None,None,None,None,None,0,0,121348.34929364058,None,347940851.73374736,0,None,0,"MULTIPOLYGON (((-78.83838 42.83207, -78.85385 ..."
4,None,District D,None,None,None,None,None,0,None,None,5,{AB3A0367-7B20-4827-B06F-B41EC47B998B},None,0,None,None,None,None,0,None,None,None,None,None,0,0,89540.395914954919,None,274432466.58708191,0,None,0,"MULTIPOLYGON (((-78.90172 42.91518, -78.90168 ..."


In [15]:
#For loop going through all the neighborhoods, checking if each coordinate is in that neighborhood and assigning a boolean:
for i in range(len(nbh_polygon)):
# mask looks at the polygon geometry of each neighborhood and maps it :
    mask =  (nbh_polygon.loc[i, 'geometry'])
# This line checks if the coordinates in crime_df are inside the mask  and outputs a boolean list:
    pip_mask_geofence = gdf.within(mask)
    gdf['geofence'] = pip_mask_geofence
# Replace all values of the new column 'neighborhood'(not neighborhood_1) with the respective neighborhood name from polygon:
    gdf.loc[gdf['geofence'] == True, 'neighborhood'] = nbh_polygon['nbhdname'][i]


gdf.sort_values('incident_datetime',inplace=True)
gdf.drop('geofence',axis=1,inplace=True)
#gdf.drop('geometry',axis=1,inplace=True)

In [16]:
#For loop going through all the neighborhoods, checking if each coordinate is in that neighborhood and assigning a boolean:
for i in range(len(cd_polygon)):
# mask looks at the polygon geometry of each neighborhood and maps it :
    mask =  (cd_polygon.loc[i, 'geometry'])
# This line checks if the coordinates in crime_df are inside the mask  and outputs a boolean list:
    pip_mask_geofence = gdf.within(mask)
    gdf['geofence'] = pip_mask_geofence
# Replace all values of the new column 'neighborhood'(not neighborhood_1) with the respective neighborhood name from polygon:
    gdf.loc[gdf['geofence'] == True, 'council_dt'] = cd_polygon['councildistrict'][i]


gdf.sort_values('incident_datetime',inplace=True)
gdf.drop('geofence',axis=1,inplace=True)
#gdf.drop('geometry',axis=1,inplace=True)

In [17]:
#For loop going through all the neighborhoods, checking if each coordinate is in that neighborhood and assigning a boolean:
for i in range(len(pd_polygon)):
# mask looks at the polygon geometry of each neighborhood and maps it :
    mask =  (pd_polygon.loc[i, 'geometry'])
# This line checks if the coordinates in crime_df are inside the mask  and outputs a boolean list:
    pip_mask_geofence = gdf.within(mask)
    gdf['geofence'] = pip_mask_geofence
# Replace all values of the new column 'neighborhood'(not neighborhood_1) with the respective neighborhood name from polygon:
    gdf.loc[gdf['geofence'] == True, 'police_dt'] = pd_polygon['name'][i]


gdf.sort_values('incident_datetime',inplace=True)
gdf.drop('geofence',axis=1,inplace=True)
gdf.drop('geometry',axis=1,inplace=True)

In [18]:
#I removed the last 5 rows as they didnt have coordinates either:
gdf[-50:-5]

,incident_id,case_number,incident_datetime,incident_type_primary,incident_description,clearance_type,address_1,address_2,city,state,zip,country,latitude,longitude,created_at,updated_at,location,hour_of_day,day_of_week,parent_incident_type,Census Tract 1,Census Block 1,Census Block Group 1,Neighborhood 1,Police District 1,Council District 1,neighborhood,police_dt,council_dt
9501,NaN,21-0480313,2021-02-16 15:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,0 Block PLYMOUTH AV,NaN,Buffalo,NY,NaN,NaN,42.900,-78.883,02/17/2021 11:42:23 AM,NaN,POINT (-78.883 42.9),11,WEDNESDAY,Theft,68,3002,3,Allentown,District B,ELLICOTT,Allentown,District B,ELLICOTT
9482,NaN,21-0470492,2021-02-16 15:25:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,0 Block SPRENGER AV,NaN,Buffalo,NY,NaN,NaN,42.911,-78.804,02/16/2021 03:28:37 PM,NaN,POINT (-78.804 42.911),15,TUESDAY,Theft,37,2001,2,Schiller Park,District C,LOVEJOY,Schiller Park,District C,LOVEJOY
9485,NaN,21-0470590,2021-02-16 15:30:00,BURGLARY,Buffalo Police are investigating this report o...,NaN,0 Block NAVEL AV,NaN,Buffalo,NY,NaN,NaN,42.918,-78.811,02/16/2021 05:20:33 PM,NaN,POINT (-78.811 42.918),17,TUESDAY,Breaking & Entering,37,4002,4,Schiller Park,District E,LOVEJOY,Schiller Park,District E,LOVEJOY
9495,NaN,21-0480161,2021-02-16 18:00:00,BURGLARY,Buffalo Police are investigating this report o...,NaN,0 Block COURT ST,NaN,Buffalo,NY,NaN,NaN,42.886,-78.875,02/17/2021 08:04:03 AM,NaN,POINT (-78.875 42.886),8,WEDNESDAY,Breaking & Entering,165,1047,1,Central,District B,ELLICOTT,Central,District B,ELLICOTT
9488,NaN,21-0470646,2021-02-16 18:20:34,ASSAULT,Buffalo Police are investigating this report o...,NaN,0 Block WESCOTT ST,NaN,Buffalo,NY,NaN,NaN,42.871,-78.830,02/16/2021 06:21:34 PM,NaN,POINT (-78.83 42.871),18,TUESDAY,Assault,163,3001,3,Seneca Babcock,District A,LOVEJOY,Seneca Babcock,District A,LOVEJOY
9489,NaN,21-0470654,2021-02-16 18:31:20,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,0 Block KILHOFFER ST,NaN,Buffalo,NY,NaN,NaN,42.913,-78.816,02/16/2021 06:32:20 PM,NaN,POINT (-78.816 42.913),18,TUESDAY,Theft,36,1012,1,Genesee-Moselle,District C,FILLMORE,Genesee-Moselle,District C,FILLMORE
9491,NaN,21-0470744,2021-02-16 20:41:26,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,1000 Block E LOVEJOY ST,NaN,Buffalo,NY,NaN,NaN,42.889,-78.808,02/16/2021 08:42:26 PM,NaN,POINT (-78.808 42.889),20,TUESDAY,Theft,23,4000,4,Lovejoy,District C,LOVEJOY,Lovejoy,District C,LOVEJOY
9492,NaN,21-0470801,2021-02-16 22:08:49,UUV,Buffalo Police are investigating this report o...,NaN,E FERRY ST & GRIDER ST,NaN,Buffalo,NY,NaN,NaN,42.914,-78.829,02/16/2021 10:09:49 PM,NaN,POINT (-78.829 42.914),22,TUESDAY,Theft of Vehicle,35,3001,3,MLK Park,District C,MASTEN,MLK Park,District C,MASTEN
9494,NaN,21-0480020,2021-02-17 00:43:35,ASSAULT,Buffalo Police are investigating this report o...,NaN,0 Block DAKOTA ST,NaN,Buffalo,NY,NaN,NaN,42.946,-78.866,02/17/2021 12:43:35 AM,NaN,POINT (-78.866 42.946),0,WEDNESDAY,Assault,49,5004,5,North Park,District D,DELAWARE,North Park,District D,DELAWARE
9616,NaN,21-0480075,2021-02-17 02:45:00,BURGLARY,Buffalo Police are investigating this report o...,NaN,200 Block SKILLEN ST,NaN,Buffalo,NY,NaN,NaN,42.961,-78.898,02/17/2021 03:18:15 AM,NaN,POINT (-78.898 42.961),3,WEDNESDAY,Breaking & Entering,58.01,1003,1,Riverside,District D,NORTH,Riverside,District D,NORTH


In [19]:
# checking null values:
gdf.isna().sum()

incident_id               12871
case_number                   0
incident_datetime             5
incident_type_primary         0
incident_description          0
clearance_type           267997
address_1                    39
address_2                267997
city                          0
state                         0
zip                      241647
country                  233708
latitude                    566
longitude                   566
created_at                    0
updated_at                12871
location                   2032
hour_of_day                   0
day_of_week                   5
parent_incident_type          0
Census Tract 1           255278
Census Block 1           255278
Census Block Group 1     255278
Neighborhood 1           255278
Police District 1        255278
Council District 1       255303
neighborhood               3421
police_dt                  3383
council_dt                 3460
dtype: int64

In [20]:
gdf.head()

,incident_id,case_number,incident_datetime,incident_type_primary,incident_description,clearance_type,address_1,address_2,city,state,zip,country,latitude,longitude,created_at,updated_at,location,hour_of_day,day_of_week,parent_incident_type,Census Tract 1,Census Block 1,Census Block Group 1,Neighborhood 1,Police District 1,Council District 1,neighborhood,police_dt,council_dt
59596,55236827.0,11-0050386,1910-12-22 00:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,200 Block CRESTWOOD AV,NaN,BUFFALO,NY,NaN,NaN,42.952764,-78.859972,01/06/2011 07:00:48 AM,10/10/2019 07:22:02 PM,POINT (-78.85997246003562 42.952764317373884),0,Thursday,Theft,NaN,NaN,NaN,NaN,NaN,NaN,North Park,District D,DELAWARE
121509,710086379.0,15-0760407,1914-12-01 22:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,100 Block LINCOLN PW,NaN,BUFFALO,NY,NaN,NaN,42.931105,-78.874554,03/18/2015 06:06:31 AM,09/23/2019 11:22:56 PM,POINT (-78.87455447748493 42.93110482058636),22,Tuesday,Theft,NaN,NaN,NaN,NaN,NaN,NaN,Elmwood Bidwell,District D,DELAWARE
204751,942663542.0,06-1840984,1951-07-03 19:51:00,UUV,Buffalo Police are investigating this report o...,NaN,200 Block W FERRY ST,NaN,BUFFALO,NY,NaN,NaN,42.915005,-78.892808,09/24/2019 10:49:30 PM,09/24/2019 10:56:37 PM,POINT (-78.892808174024 42.915005299524026),19,Tuesday,Theft of Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,Upper West Side,District B,NIAGARA
146812,146621859.0,13-0730379,1951-12-05 02:20:21,ASSAULT,Buffalo Police are investigating this report o...,NaN,200 Block CAMBRIDGE AV,NaN,BUFFALO,NY,NaN,NaN,42.921123,-78.822961,03/15/2013 06:00:48 AM,03/21/2013 06:00:42 AM,POINT (-78.822961 42.921123),2,Wednesday,Assault,NaN,NaN,NaN,NaN,NaN,NaN,Genesee-Moselle,District E,MASTEN
233717,942606470.0,08-3530528,1952-08-30 16:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,200 Block JEFFERSON AV,NaN,BUFFALO,NY,NaN,NaN,42.886733,-78.853599,09/24/2019 10:32:01 PM,09/24/2019 10:34:37 PM,POINT (-78.85359911134378 42.88673286341301),16,Saturday,Theft,NaN,NaN,NaN,NaN,NaN,NaN,Broadway Fillmore,District C,ELLICOTT


In [21]:
gdf.to_csv('crime.csv')